Possible examples/exercise:

* one-hot-vector type
* point type
* volume type (constructor should check for real positive values)

Maybe mention type piracy at the bottom?

# Defining data types

It is a convention that type names are capitalized and [camel cased](https://en.wikipedia.org/wiki/Camel_case). Also, note that types can not be redefined - you have to restart your Julia session to change a type definiton.

In [135]:
struct MyType end

In [136]:
m = MyType()

MyType()

In [137]:
typeof(m)

MyType

In [138]:
f(x::MyType) = println("I'm the only method that works with objects of MyType.")

f (generic function with 1 method)

In [139]:
f(m)

I'm the only method that works with objects of MyType.


In [141]:
struct A
    x::Int64
end

In [143]:
A()

MethodError: MethodError: no method matching A()
Closest candidates are:
  A(!Matched::Int64) at In[141]:2
  A(!Matched::Any) at In[141]:2

In [145]:
A(3)

A(3)

In [147]:
a = A(3)

A(3)

In [150]:
# a.<TAB>
a.x

3

Note that types defined with `struct` are **immutable**, that is the values of it's fields cannot be changed.

In [152]:
a.x = 2

ErrorException: setfield! immutable struct of type A cannot be changed

In [153]:
mutable struct B
    x::Int64
end

In [154]:
b = B(3)

B(3)

In [155]:
b.x

3

In [156]:
b.x = 4

4

In [157]:
b.x

4

Note, however, that **immutability is not recursive**.

In [159]:
struct C
    x::Vector{Int64}
end

In [160]:
c = C([1, 2, 3])

C([1, 2, 3])

In [161]:
c.x

3-element Array{Int64,1}:
 1
 2
 3

In [162]:
c.x = [3,4,5]

ErrorException: setfield! immutable struct of type C cannot be changed

In [163]:
c.x[1] = 3

3

In [164]:
c.x

3-element Array{Int64,1}:
 3
 2
 3

In [165]:
c.x .= [3,4,5] # dot to perform the assignment element-wise

3-element Array{Int64,1}:
 3
 4
 5

Abstract types are just as easy.

In [167]:
abstract type MyAbstractType end

In [168]:
struct MyConcreteType <: MyAbstractType end

In [169]:
c = MyConcreteType()

MyConcreteType()

In [170]:
c isa MyAbstractType

true

As noted above, the only purpose of abstract types is to define an interface (a common behavior) for a bunch of concrete types. This focus on *behavior* rather than *structure* is known as **duck typing**.

# Interlude: Benchmarking with `BenchmarkTools.jl`

In [1]:
using BenchmarkTools

In [2]:
operation(x) = x^2 + sqrt(x)

operation (generic function with 1 method)

In [3]:
x = rand(2,2)
@time operation.(x)

  0.092506 seconds (352.53 k allocations: 17.787 MiB, 6.52% gc time)


2×2 Array{Float64,2}:
 0.400098  1.69656 
 1.9004    0.323193

In [4]:
function f()
    x = rand(2,2)
    @time operation.(x)
end

f (generic function with 1 method)

In [5]:
f()

  0.000000 seconds (1 allocation: 112 bytes)


2×2 Array{Float64,2}:
 1.20004   0.620318
 0.179407  1.89603 

We should wrap benchmarks into functions!

Fortunately, there are tools that do this for us. In addition, they also collect some statistics by running the benchmark multiple times.

In [6]:
@benchmark operation.(x)

BenchmarkTools.Trial: 
  memory estimate:  144 bytes
  allocs estimate:  3
  --------------
  minimum time:     243.645 ns (0.00% GC)
  median time:      260.432 ns (0.00% GC)
  mean time:        320.715 ns (6.12% GC)
  maximum time:     101.181 μs (99.71% GC)
  --------------
  samples:          10000
  evals/sample:     417

Typically we don't need all this information. Just use `@btime` instead of `@time`!

In [7]:
@btime operation.(x);

  246.173 ns (3 allocations: 144 bytes)


However, we still have to take some care to avoid accessing global variables.

In [8]:
@btime operation.($x); # interpolate the value of x into the expression to avoid overhead of globals

  32.429 ns (1 allocation: 112 bytes)


More information: [BenchmarkTools.jl](https://github.com/JuliaCI/BenchmarkTools.jl/blob/master/doc/manual.md).